In [1]:
!pip install pyspark

In [2]:
# Imports
import os
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from google.colab import drive

In [3]:
# Entry Point to Spark
spark = (SparkSession
         .builder
         .master('local[*]')
         .appName('Spark')
         .getOrCreate())

In [4]:
# Google Drive connection
drive.mount('/content/drive', force_remount=True)
path = '/content/drive/MyDrive/cnpj/2024/_01JAN2024_31DEZ2024.csv'

Mounted at /content/drive


In [5]:
# Reading CSV file as Spark DataFrame
df_pd = pd.read_csv(path, sep=",")
df = (
    spark
    .createDataFrame(df_pd)
    .drop("Identificador")
    .withColumnRenamed("Descrição", "Descricao")
)

In [6]:
df_ajusted = (df
            .withColumn("Descricao", upper(substr("Descricao", lit(34), lit(4))))
            .withColumn("Data", date_format(to_date("Data", "dd/MM/yyyy"), "yyyyMM"))
            .withColumn("Valor", regexp_replace("Valor","-","").cast("double"))
            )

In [23]:
# Filter rows
df_select = df_ajusted.filter("Descricao == 'FABR'")

In [31]:
# Group and sum rows
df_month = df_select.groupby("Data").agg(sum("Valor")).orderBy(asc("Data"))

In [33]:
# Total sum
total = df_select.agg(sum("Valor")).first()[0]